In [1]:
import time
import itertools
import numpy as np
from package import geometry

In [2]:
cube2d = np.array(list(itertools.product([1, -1], repeat=2)))
cube3d = np.array(list(itertools.product([1, -1], repeat=3)))
cube4d = np.array(list(itertools.product([1, -1], repeat=4)))

truncated_octahedron = np.array(list(itertools.permutations([0, 1, 2], 3)))
truncated_octahedron = np.concatenate([truncated_octahedron*np.array(i) for i in itertools.product([1, -1], repeat=3)])
truncated_octahedron = 0.5*np.unique(truncated_octahedron, axis=0)

figures = {'cube2d': cube2d, 
           'cube3d': cube3d, 
           'TrOcth' : truncated_octahedron, 
           'cube4d': cube4d, }

for key in figures:
    print(f'{key}: Dimension: {figures[key].shape[1]}; Vertices: {figures[key].shape[0]}')

cube2d: Dimension: 2; Vertices: 4
cube3d: Dimension: 3; Vertices: 8
TrOcth: Dimension: 3; Vertices: 24
cube4d: Dimension: 4; Vertices: 16


# Geometry

In [3]:
matrices = {}
hspaces = {}
for key in figures:
    figure = figures[key]
    clock = time.perf_counter()
    m_list, h_list = geometry.get_hyperspaces_containing(figure)
    clock = time.perf_counter() - clock
    matrices.update({key : m_list.copy()})
    hspaces.update({key : [i for i in h_list]})
    print("Hyperspcaes for {0} has been calculated in {1:.4f} seconds.".format(key, clock))

print()
for key in hspaces:
    lengths = np.array([len(hspace) for hspace in hspaces[key]])
    length2 = []
    for length in np.unique(lengths):
        length2.append(f"Hyperspaces size {length} : {(lengths == length).sum()}")
    msg = f"Figure {key}\nHyperspaces total: {len(lengths)}\n" + '\n'.join(length2) + '\n'
    print(msg)

Hyperspcaes for cube2d has been calculated in 0.0010 seconds.
Hyperspcaes for cube3d has been calculated in 0.0061 seconds.
Hyperspcaes for TrOcth has been calculated in 0.0750 seconds.
Hyperspcaes for cube4d has been calculated in 0.1031 seconds.

Figure cube2d
Hyperspaces total: 2
Hyperspaces size 2 : 2

Figure cube3d
Hyperspaces total: 6
Hyperspaces size 4 : 6

Figure TrOcth
Hyperspaces total: 51
Hyperspaces size 4 : 48
Hyperspaces size 8 : 3

Figure cube4d
Hyperspaces total: 20
Hyperspaces size 6 : 8
Hyperspaces size 8 : 12



In [4]:
halfs = {}
for key in figures:
    figure = np.array(figures[key])
    hspace = hspaces[key]
    lengths = np.array([len(hspc) for hspc in hspace])
    matrix = matrices[key][np.where(lengths == max(lengths))[0][0]]
    clock = time.perf_counter()
    half = geometry.get_half(figure, matrix)
    clock = time.perf_counter() - clock
    halfs.update({key : half})
    print("Half for {0} has been calculated in {1:.4f} seconds.".format(key, clock))
    
print('\nHalfs I got:')
for key in halfs:
    print(f'{key}: Dimension: {halfs[key].shape[1]}; Vertices: {halfs[key].shape[0]}')
    
assert [halfs[key].shape[0] for key in halfs] == [3, 6, 16, 12]

Half for cube2d has been calculated in 0.0006 seconds.
Half for cube3d has been calculated in 0.0007 seconds.
Half for TrOcth has been calculated in 0.0005 seconds.
Half for cube4d has been calculated in 0.0007 seconds.

Halfs I got:
cube2d: Dimension: 2; Vertices: 3
cube3d: Dimension: 3; Vertices: 6
TrOcth: Dimension: 3; Vertices: 16
cube4d: Dimension: 4; Vertices: 12


In [5]:
normals = {}
for key in figures:
    figure = figures[key]
    matrices_key = matrices[key]
    normals_key = []
    clock = time.perf_counter()
    for matrix in matrices_key:
        normals_matrix = geometry.get_normal(matrix)
        normals_key.append(normals_matrix)
    clock = time.perf_counter() - clock
    normals.update({key : np.array(normals_key)})
    print("All normals for hyperspaces in {0} has been calculated in {1:.4f} seconds.".format(key, clock))
    
print()
for key in normals:
    print(f'normals[{key}].shape = {normals[key].shape}')

All normals for hyperspaces in cube2d has been calculated in 0.0004 seconds.
All normals for hyperspaces in cube3d has been calculated in 0.0006 seconds.
All normals for hyperspaces in TrOcth has been calculated in 0.0036 seconds.
All normals for hyperspaces in cube4d has been calculated in 0.0011 seconds.

normals[cube2d].shape = (2, 2)
normals[cube3d].shape = (6, 3)
normals[TrOcth].shape = (51, 3)
normals[cube4d].shape = (20, 4)


In [6]:
refs = {}
for key in figures:
    figure = figures[key]
    hspace = hspaces[key]
    lengths = np.array([len(hspc) for hspc in hspace])
    normal = normals[key][np.where(lengths == max(lengths))[0][0]]
    half = halfs[key]
    
    clock = time.perf_counter()
    ref = geometry.reflect(half, normal)
    clock = time.perf_counter() - clock
    refs.update({key : ref})
    print("Half for {0} has been reflected in {1:.4f} seconds.".format(key, clock))

print('\nReflections of halfs I got:')
for key in halfs:
    print(f'{key}: Dimension: {refs[key].shape[1]}; Vertices: {refs[key].shape[0]}')
    
assert [refs[key].shape[0] for key in refs] == [halfs[key].shape[0] for key in halfs]

Half for cube2d has been reflected in 0.0001 seconds.
Half for cube3d has been reflected in 0.0001 seconds.
Half for TrOcth has been reflected in 0.0000 seconds.
Half for cube4d has been reflected in 0.0000 seconds.

Reflections of halfs I got:
cube2d: Dimension: 2; Vertices: 3
cube3d: Dimension: 3; Vertices: 6
TrOcth: Dimension: 3; Vertices: 16
cube4d: Dimension: 4; Vertices: 12


In [7]:
half_unions = {}
for key in figures:
    half_unions.update({key : np.unique(np.concatenate([halfs[key], refs[key]]), axis=0)})
    
print('Unions of halfs')
for key in half_unions:
    print(f'{key}: Dimension: {half_unions[key].shape[1]}; Vertices: {half_unions[key].shape[0]}')
    
for key in figures:
    try:
        assert (np.sort(figures[key], axis=0) == np.sort(half_unions[key], axis=0)).all()
    except AssertionError:
        print()
        print(f'Something is wrong for figure {key}:')
        print(f'figures[{key}]:\n{figures[key]}')
        print(f'half_unions[{key}]:\n{half_unions[key]}')

Unions of halfs
cube2d: Dimension: 2; Vertices: 4
cube3d: Dimension: 3; Vertices: 8
TrOcth: Dimension: 3; Vertices: 24
cube4d: Dimension: 4; Vertices: 16


# Triangulation